In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
import json

In [6]:
panel3_task_year_df = pd.read_csv("../static/data/csv/Panel 3.csv")
panel3_job_skill_2008_df = pd.read_csv("../static/data/csv/Panel 3 Skill Importance 2008.csv")
panel3_job_skill_2019_df = pd.read_csv("../static/data/csv/Panel 3 Skill Importance 2019.csv")
mst_edges = pd.read_excel("../static/data/xlsx/MST Edges.xlsx")

In [7]:
panel3_job_skill_2019_df.head()

,Job Code,Job Title,Active Learning,Wrist-Finger Speed,Number Facility,Electronic Mail,In an Open Vehicle or Equipment,"Guiding, Directing, and Motivating Subordinates",Exposed to High Places,"Inspecting Equipment, Structures, or Material",...,Negotiation,Exposed to Whole Body Vibration,Responsibility for Outcomes and Results,Critical Thinking,Documenting/Recording Information,Pace Determined by Speed of Equipment,Impact of Decisions on Co-workers or Company Results,Sound Localization,Selling or Influencing Others,Finger Dexterity
0,00-0000,TOTAL AVERAGE,0.005153,0.006004,0.006238,0.007199,0.004171,0.004526,0.004076,0.005198,...,0.005543,0.003324,0.004511,0.005597,0.005817,0.00592,0.004848,0.005148,0.006031,0.005871
1,11-1011,Chief Executives,0.011427,0.000000,0.008704,0.005994,0.000289,0.017453,0.000000,0.000000,...,0.018997,0.002607,0.012023,0.012390,0.000000,0.00000,0.013786,0.000000,0.011320,0.000000
2,11-1021,General and Operations Managers,0.008371,0.000000,0.004497,0.008413,0.000000,0.021092,0.002898,0.003211,...,0.020354,0.000000,0.020745,0.008813,0.001589,0.00000,0.014911,0.001704,0.016846,0.000000
3,11-2011,Advertising and Promotions Managers,0.005194,0.000000,0.010406,0.014330,0.000000,0.003663,0.000000,0.000000,...,0.015706,0.000000,0.013109,0.017165,0.000000,0.00000,0.006843,0.000000,0.026177,0.000000
4,11-2021,Marketing Managers,0.017921,0.000000,0.012927,0.010800,0.000000,0.018276,0.000000,0.000000,...,0.023033,0.000000,0.014231,0.009973,0.000000,0.00000,0.007969,0.000000,0.030220,0.000000


In [8]:
def standardizer(df, col):
    df[col] -= df[col].min()
    df[col] /= df[col].max()
    
    df[col] = (30**df[col]-1)/30
    
    df[col] = [0.5 if value > 0.5 else value for value in df[col]]
    
    #df[col] = df[col]*200
    #df[col] = df[col]+10
    return df

In [9]:
def standardizer_diff(df, col):
    df[col] -= df[col].min()
    df[col] /= df[col].max()
    
    df[col] = (30**df[col]-1)/30
    
    df[col] = [0.6 if value > 0.6 else value for value in df[col]]
    
    #df[col] = df[col]*200
    #df[col] = df[col]+10
    return df

In [10]:
panel3_task_year_df = standardizer(panel3_task_year_df, "2008 Automation Level")
panel3_task_year_df = standardizer(panel3_task_year_df, "2019 Automation Level")
panel3_task_year_df = standardizer_diff(panel3_task_year_df, "Change during 2008-2019")

In [11]:
panel3_task_year_df.head()

,Skill Id,Skill Name,Task,2019 Automation Level,2008 Automation Level,Change during 2008-2019
0,4.A.3.b.1,Interacting With Computers,Administrative Task and Communication,0.299335,0.309173,0.069780
1,2.B.3.e,Programming,Administrative Task and Communication,0.250588,0.153256,0.278059
2,4.C.2.d.1.a,Spend Time Sitting,Administrative Task and Communication,0.216880,0.202520,0.090418
3,4.A.4.c.1,Performing Administrative Activities,Administrative Task and Communication,0.175269,0.118865,0.196262
4,4.C.1.a.2.j,Letters and Memos,Administrative Task and Communication,0.146159,0.082767,0.270968


# Skill Network Edges

In [12]:
mst_edges.to_csv("../static/data/network_csv/skill_edges.csv", index=None)

# Skill Network Nodes

In [13]:
panel3_task_year_df_gp = panel3_task_year_df.groupby('Task')

In [14]:
for i in range(panel3_job_skill_2019_df.shape[0]):    
    skillmap = {}
    skillmap["job_title"] = panel3_job_skill_2019_df.iloc[i]['Job Title']
    skillmap["job_code"] = panel3_job_skill_2019_df.iloc[i]['Job Code']  
    
    skill_list = []
    
    for nm, gp in panel3_task_year_df_gp:    
        for j, r2 in gp.iterrows():
            skill_dict = {"skill_name": r2["Skill Name"],
                          "task_name": nm,
                          #"skill_importance_2019": panel3_job_skill_2019_df.iloc[i][r2["Skill Name"]],
                          #"skill_importance_2008":panel3_job_skill_2008_df.iloc[i][r2["Skill Name"]], 
                          #"skill_importance_2008_2019": panel3_job_skill_2019_df.iloc[i][r2["Skill Name"]],
                          "automation_2019":r2["2019 Automation Level"],
                          "automation_2008":r2["2008 Automation Level"],                                     
                          "automation_2008_2019":r2["Change during 2008-2019"],
                         }
            skill_list.append(skill_dict)  

    skillmap["skill_list"] = skill_list
        
    #with open('../static/data/network/'+r1['Job Code']+'-2019'+'.json', 'w') as outfile:
    with open('../static/data/network/'+panel3_job_skill_2019_df.iloc[i]['Job Code']+'.json', 'w') as outfile:
        json.dump(skillmap, outfile)

# Task Clusters

In [15]:
import pandas as pd

In [16]:
task_cluster = {}

task_cluster['Race with the Machine'] = ['Interpersonal Task', 'Administrative Task and Communication']

task_cluster['Race against the Machine'] = ['Information Processing', 'Routine Problem Solving',
                                           'Vehicle and Equipment Operation', 'Machine Control',
                                           'Equipment Maintenance', 'General Physical Task']

task_cluster['Race ahead of the Machine'] = ['Systems Thinking', 'Creative and Critical Thinking']

task_cluster['Running a Different Race'] = ['Dynamic Physical Task', 'Hazardous and Group Task',
                                           'Conflict Resolution', 'Management and Supervision']

In [17]:
task_cluster

{'Race with the Machine': ['Interpersonal Task',
  'Administrative Task and Communication'],
 'Race against the Machine': ['Information Processing',
  'Routine Problem Solving',
  'Vehicle and Equipment Operation',
  'Machine Control',
  'Equipment Maintenance',
  'General Physical Task'],
 'Race ahead of the Machine': ['Systems Thinking',
  'Creative and Critical Thinking'],
 'Running a Different Race': ['Dynamic Physical Task',
  'Hazardous and Group Task',
  'Conflict Resolution',
  'Management and Supervision']}

In [18]:
flatten_task_cluster = []

for task_cluster_nm, tasks in task_cluster.items():
    for task in tasks:
        flatten_task_cluster_dict = {}
        flatten_task_cluster_dict['task_name'] = task
        flatten_task_cluster_dict['task_cluster'] = task_cluster_nm
        flatten_task_cluster.append(flatten_task_cluster_dict)

In [19]:
task_cluster_df = pd.DataFrame(flatten_task_cluster)

In [20]:
task_cluster_df.to_csv("../static/data/network_csv/task_cluster.csv", index=None)